## Setup

In [ ]:
!pip install -qqq torch

In [1]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("jiangfeilong/sun-attribute")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sun-attribute


In [2]:
import subprocess
try:
    subprocess.run(["cp", "-r", "/root/.cache/kagglehub/datasets/jiangfeilong/sun-attribute/versions/1", "."], check=True)
except Exception as e:
    subprocess.run(["cp", "-r", "/kaggle/input/sun-attribute", "."], check=True)

In [3]:
import os
try:
    os.chdir("1/SUN/SUNAttributeDB/")
except FileNotFoundError:
    os.chdir("sun-attribute/SUN/SUNAttributeDB/")

In [4]:
# @title Extract attribute labels for images
import scipy.io
import numpy as np

# Load the .mat file
images_paths = scipy.io.loadmat('images.mat')
images_paths = images_paths['images']

labels_attr = scipy.io.loadmat('attributeLabels_continuous.mat')
labels_attr = labels_attr['labels_cv']
print(len(labels_attr))  # If it's an array

attribute_names = scipy.io.loadmat('attributes.mat')
attribute_names = attribute_names['attributes']

print(len(attribute_names))

14340
102


In [5]:
# @title Create Image Metadata

def create_list_of_dictionary_dataset(images_data, labels_attr):
  dataset_list = []
  for i in range(len(images_data)):
    data_dict = dict()
    path = str(images_data[i][0][0])

    data_dict['path'] = path
    splitted_path = path.split('/')
    if len(splitted_path) == 3:
      category = splitted_path[-2]
    elif len(splitted_path) == 4:
      category = '_'.join(splitted_path[-3:-1])
      # print(category)
    else:
      print("error")

    data_dict['label'] = category
    data_dict['labels_attr'] = labels_attr[i]

    dataset_list.append(data_dict)
  return dataset_list


images_metadata = create_list_of_dictionary_dataset(images_paths, labels_attr)

In [6]:
%cd ../images

/content/sun-attribute/SUN/images


In [7]:
# @title Create Test Set
import os
import shutil

moved_files_set = set()

test_images_dir = "../test_images"
# Create the test_images directory if it doesn't exist
if not os.path.exists(test_images_dir):
    os.mkdir(test_images_dir)

def extract_test_set(root_path):
    """
    Extracts a percentage of files from a nested directory structure
    into a new 'test_images' directory, preserving category names.

    It moves files from third-level directories if they exist.
    If no third-level directories exist within a second-level directory,
    it moves files directly from the second-level directory.

    Args:
        root_path (str): The root directory to start scanning from.

    Returns:
        int: The total number of files moved.
    """
    number_files_moved = 0

    # Iterate through the first level directories
    for first_level in os.listdir(root_path):
        first_level_path = os.path.join(root_path, first_level)

        # Ensure it's a directory before proceeding
        if os.path.isdir(first_level_path):
            if (first_level == "misc" or first_level == "outliers"): continue;
            # Iterate through the second level directories within the first level
            for second_level in os.listdir(first_level_path):
                second_level_path = os.path.join(first_level_path, second_level)
                second_level_category = second_level # Use second level name for base category

                # Ensure it's a directory before proceeding
                if os.path.isdir(second_level_path):

                    # Check if there are any third-level directories within the second level
                    third_level_dirs = [
                        d for d in os.listdir(second_level_path)
                        if os.path.isdir(os.path.join(second_level_path, d))
                    ]

                    # --- Logic to decide whether to process second or third level ---

                    if third_level_dirs:
                        # Process files in the third level directories if they exist
                        for third_level in third_level_dirs:
                            third_level_path = os.path.join(second_level_path, third_level)
                            # Create category name combining second and third levels
                            third_level_category_name = f"{second_level_category}_{third_level}"

                            # Get list of files in the third level directory
                            third_level_files = [
                                f for f in os.listdir(third_level_path)
                                if os.path.isfile(os.path.join(third_level_path, f))
                            ]

                            if third_level_files:
                                # Create the destination category directory
                                category_dir = os.path.join(test_images_dir, third_level_category_name)
                                if not os.path.exists(category_dir):
                                    os.mkdir(category_dir)

                                # Determine number of files to move (at least 1, or 10%)
                                number_files_to_move = max(int(len(third_level_files) * 0.1), 1)

                                # Move the selected files
                                for file in third_level_files[:number_files_to_move]:
                                    file_to_move = os.path.join(third_level_path, file)
                                    destination_path = os.path.join(category_dir, file) # Define full destination path
                                    try:
                                        shutil.move(file_to_move, destination_path)
                                        moved_file = file_to_move[2:]
                                        moved_files_set.add(moved_file) # Add destination path to set
                                        number_files_moved += 1 # Increment count for each file moved
                                        # print(f"Moved: {file_to_move} -> {destination_path}")
                                    except Exception as e:
                                        print(f"Error moving file {file_to_move}: {e}")

                    else:
                        # Process files directly in the second level if NO third-level directories exist
                        second_level_files = [
                            f for f in os.listdir(second_level_path)
                            if os.path.isfile(os.path.join(second_level_path, f))
                        ]

                        if second_level_files:
                            # Create the destination category directory using the second level name
                            category_dir = os.path.join(test_images_dir, second_level_category)
                            if not os.path.exists(category_dir):
                                os.mkdir(category_dir)

                            # Determine number of files to move (at least 1, or 10%)
                            number_files_to_move = max(int(len(second_level_files) * 0.1), 1)

                            # Move the selected files
                            for file in second_level_files[:number_files_to_move]:
                                file_to_move = os.path.join(second_level_path, file)
                                destination_path = os.path.join(category_dir, file) # Define full destination path
                                try:
                                    shutil.move(file_to_move, destination_path)

                                    moved_file = file_to_move[2:]
                                    moved_files_set.add(moved_file)
                                    number_files_moved += 1 # Increment count for each file moved
                                    # print(f"Moved: {file_to_move} -> {destination_path}")
                                except Exception as e:
                                    print(f"Error moving file {file_to_move}: {e}")

    return number_files_moved


number_files_moved = extract_test_set(".")

print(f"\nTotal files moved: {number_files_moved}")


Total files moved: 1440


In [8]:
%cd ../test_images

/content/sun-attribute/SUN/test_images


In [9]:
# @title Validating testset creation and removing directories of one element
# images_paths = scipy.io.loadmat('att_splits.mat')

# allclasses_ds = set()
# for allclasses in images_paths['allclasses_names']:
#   # print(allclasses[0][0])
#   allclasses_ds.add(allclasses[0][0])

# # images_paths.__getitem__('train_loc').shape

import os

def count_directories(directory_path="."):
  """
  Counts the number of subdirectories within a given directory.

  Args:
    directory_path (str): The path to the directory to examine.
                          Defaults to the current directory (.).

  Returns:
    int: The number of directories found, or -1 if the directory doesn't exist
         or is not accessible.
  """
  empty_dir = {"roller_skating_rink_indoor", "police_station", "volleyball_court_indoor", "distillery", "barbershop", "ice_cream_parlor"}
  count = 0
  try:
    # Use os.scandir for better performance and getting file type info directly
    with os.scandir(directory_path) as entries:
      for entry in entries:

        theEntry = entry.name
        if (theEntry in empty_dir): shutil.rmtree(theEntry); continue
        # Check if the entry is a directory (and not a symbolic link to a directory)
        if entry.is_dir() and not entry.is_symlink():
          count += 1
  except FileNotFoundError:
    print(f"Error: Directory not found at '{directory_path}'")
    return -1
  except PermissionError:
    print(f"Error: Permission denied to access '{directory_path}'")
    return -1
  except Exception as e:
    print(f"An unexpected error occurred: {e}")
    return -1

  return count

# --- Example Usage ---

# Count directories in the current directory
current_dir_count = count_directories(".")
if current_dir_count != -1:
  print(f"Number of directories in the current directory: {current_dir_count}")


Number of directories in the current directory: 717


In [10]:
# @title Images labels for Dataloader
images_labels = {example['path']:example['labels_attr'].tolist() for example in images_metadata if (example['path']) not in moved_files_set}

In [11]:
print(len(images_labels))

12940


## Models

In [ ]:
# import os

# loc_dict = dict()
# def traverse_directory(root_dir):
#     # print("hello")
#     for dirpath, dirnames, filenames in os.walk(root_dir):
#         # print(filenames)
#         # print(dirpath)
#         # Skip the root directory itself
#         for d in dirnames:
#           # print(d)
#           for dirpath, dirnames, filenames in os.walk(os.path.join(root_dir,d)):
#             if dirpath == root_dir:
#                 continue

#             # Print the current directory path
#             # print(f"\n{dirpath}:")

#             # Print all files in this directory
#             for filename in sorted(filenames):
#                 # print(f"  {filename}")
#                 loc_dict[filename] = dirpath.split('/')[-1]

# # Example usage:
# if __name__ == "__main__":
#     directory_to_scan = "./1/SUN/images"  # Change this to your directory
#     traverse_directory(directory_to_scan)

# print((loc_dict))

# Extract Dense Features of an image

In [12]:
# @title Extract Dense Features P1
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import os

# Initialize model
alexnet = models.alexnet(weights='AlexNet_Weights.DEFAULT')
feature_extractor = nn.Sequential(*list(alexnet.classifier.children())[:-2])

class CNNFeatureExtractor(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.features = model.features
        self.avgpool = model.avgpool
        self.classifier = feature_extractor

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x  # Shape: [batch_size, 4096]

# Define transforms
alexnet_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Initialize feature extractor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNFeatureExtractor(alexnet).to(device)
model.eval()

def process_image(image_path):
    """Process a single image and return features with label"""
    image = Image.open(image_path).convert("RGB")
    img_tensor = alexnet_transforms(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(img_tensor)  # shape: [1, 4096]
        return features.cpu().numpy().flatten()

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 163MB/s]


In [ ]:
%cd ../images

/content/1/SUN/images


In [ ]:
# @title Extract Dense Features P2

images_deep_features = list()

# Process a single image
# image_path = "./1/SUN/images/a/abbey/sun_aacphuqehdodwawg.jpg"
# label = "abbey"
# label, features = process_image(image_path, label)
# features_dict[label] = features

i = 0
for d in images_metadata:
  label = d['label']
  image_path = os.path.join('.', d['path'])
  try:
    features = process_image(image_path)
  except:
    pass

  features_dict = dict()
  # features_dict[image_path] =
  images_deep_features.append({
    'path': image_path,
    'label': label,
    'features_vector': features
  })

  i += 1
  print("Reached: ", i)
  if (i == 500):
    break

## New Approach

In [17]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

class AttributePredictor(nn.Module):
    def __init__(self, num_attributes=102):
        super().__init__()
        self.backbone = models.resnet152(pretrained=True)
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_attributes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.backbone(x))

# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_res = AttributePredictor(num_attributes=102).to(device)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for multi-label
optimizer = torch.optim.Adam(model_res.parameters(), lr=0.001)

# Data preparation (example)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Training function
def train(model, dataloader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item()
            predicted = (outputs > 0.5).int()
            total += labels.size(0) * labels.size(1)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(dataloader)
        epoch_acc = correct / total
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

# Evaluation function
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            predicted = (outputs > 0.5).int()
            total += labels.size(0) * labels.size(1)
            correct += (predicted == labels).sum().item()

    test_loss = running_loss / len(dataloader)
    test_acc = correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

    return test_loss, test_acc

# Inference for a single image
def predict_attributes(model, image_path, transform):
    from PIL import Image

    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_tensor)
        predicted_attributes = (output > 0.5).int()  # Binarize to 0/1

    return predicted_attributes

# Example usage:
# train(model, train_dataloader, criterion, optimizer, epochs=10)
# test_loss, test_acc = evaluate(model, test_dataloader, criterion)
# predicted_attrs = predict_attributes(model, "path/to/image.jpg", transform)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:01<00:00, 172MB/s]


In [ ]:
# @title Training and Evaluation
train(model, train_dataloader, criterion, optimizer, epochs=40)
evaluate(model, val_dataloader, criterion)

In [ ]:
!pwd

/content/1/SUN/images


In [ ]:
query_vector_example = process_image('../test_images/abbey/sun_ajblooupyqwdvzgx.jpg')

In [ ]:
features = process_image('../test_images/airport_entrance/sun_asimhuxhlemcrozs.jpg')
zero_percentage = (features == 0).sum() / features.size * 100
print(f"Percentage of zeros: {zero_percentage:.2f}%")

Percentage of zeros: 0.00%


In [ ]:
print(features[1990])

-14.041405


## Create DataLoaders and Datasets

In [13]:
from torch.utils.data import Dataset
from PIL import Image
import os
import torch

class AttributeDataset(Dataset):
    def __init__(self, image_dir, label_dict, transform=None):
        self.image_dir = image_dir
        self.label_dict = label_dict
        self.image_files = list(label_dict.keys())
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.label_dict[img_name], dtype=torch.float32)
        return image, label


In [21]:
%cd ../images

/content/sun-attribute/SUN/images


In [14]:
print(len(images_labels))

12940


In [22]:
# Set this to your actual folder
image_dir = "."

# Use your transform (same as the one you already defined)
dataset = AttributeDataset(image_dir=image_dir, label_dict=images_labels, transform=transform)

# Create train/test splits if needed
from torch.utils.data import random_split

train_size = int(0.99 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [23]:
# Assuming train_dataset is the Subset object created by random_split
num_train_samples = len(train_dataset)

print(f"The number of elements in the training dataset is: {num_train_samples}")


num_train_samples = len(val_dataset)

print(f"The number of elements in the training dataset is: {num_train_samples}")

The number of elements in the training dataset is: 10352
The number of elements in the training dataset is: 2588


In [24]:
!ls

a  c  e  g  i  k  m	n  outliers  q	s  u  w  z
b  d  f  h  j  l  misc	o  p	     r	t  v  y


In [25]:
print("\n--- Content of one batch from train_dataloader ---")
if len(train_dataloader) > 0:
    # Get one batch of data
    batch_images, batch_labels = next(iter(train_dataloader)) # Assuming dataloader yields (images, labels)

    print(f"Batch Image Tensor Shape: {batch_images.shape}") # e.g., [32, C, H, W]
    print(f"Batch Labels Shape: {batch_labels.shape}")     # e.g., [32] or [32, num_attributes]
    print(f"Sample Labels from Batch: {batch_labels[:1]}...") # Show first 5 labels in the batch
else:
    print("Train dataloader is empty or could not iterate.")


--- Content of one batch from train_dataloader ---
Batch Image Tensor Shape: torch.Size([32, 3, 224, 224])
Batch Labels Shape: torch.Size([32, 102])
Sample Labels from Batch: tensor([[0.0000, 0.0000, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.6667, 0.0000, 0.6667, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6667, 0.0000,
         0.000

In [ ]:
# @title deprecated
# import os
# import torch
# import numpy as np
# from PIL import Image
# from torch.utils.data import Dataset, DataLoader, random_split
# from torchvision import transforms
# import glob

# class SUNDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         """
#         Args:
#             root_dir (string): Root directory with images organized in subdirectories by class
#             transform (callable, optional): Optional transform to be applied on images
#         """
#         self.root_dir = root_dir
#         self.transform = transform
#         self.classes = []
#         self.class_to_idx = {}
#         self.images = []
#         self.labels = []

#         # Find all class directories (scene categories)
#         for path, dirs, files in os.walk(root_dir):
#             # Get only the innermost directories that contain images
#             if files and any(f.lower().endswith(('.jpg', '.jpeg', '.png')) for f in files):
#                 # Extract class name from path
#                 class_name = os.path.basename(path)

#                 # Add to class list if not already there
#                 if class_name not in self.class_to_idx:
#                     self.class_to_idx[class_name] = len(self.classes)
#                     self.classes.append(class_name)

#                 # Add all images from this class
#                 for img_file in files:
#                     if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
#                         img_path = os.path.join(path, img_file)
#                         self.images.append(img_path)
#                         self.labels.append(self.class_to_idx[class_name])

#         print(f"Found {len(self.classes)} classes and {len(self.images)} images")

#     def __len__(self):
#         return len(self.images)

#     def __getitem__(self, idx):
#         img_path = self.images[idx]
#         image = Image.open(img_path).convert('RGB')
#         label = self.labels[idx]

#         if self.transform:
#             image = self.transform(image)

#         # For multi-class classification, use a one-hot encoded vector
#         # For the AttributePredictor model you showed earlier
#         one_hot = torch.zeros(len(self.classes))
#         one_hot[label] = 1.0

#         return image, one_hot

# # Alternative implementation for multi-label scenarios where images might have multiple attributes
# class SUNMultiLabelDataset(Dataset):
#     def __init__(self, root_dir, images_metadata, attributes_file=None, transform=None):
#         """
#         Args:
#             root_dir (string): Root directory with images
#             attributes_file (string): Path to a file mapping images to multiple attributes
#             transform (callable, optional): Optional transform to be applied on images
#         """
#         self.root_dir = root_dir
#         self.transform = transform
#         self.images = []
#         self.attributes = set()

#         for image_metadata in images_metadata:
#           self.images.append(image_metadata['path'])
#           self.attr
#         # Find all image files
#         # self.images = []
#         # for ext in ['*.jpg', '*.jpeg', '*.png']:
#         #     self.images.extend(glob.glob(os.path.join(root_dir, '**', ext), recursive=True))

#         # # Get attribute names from directory structure
#         # self.attributes = set()
#         # for path in self.images:
#         #     # Extract innermost directory name as attribute
#         #     attr = os.path.basename(os.path.dirname(path))
#         #     self.attributes.add(attr)

#         # self.attributes = sorted(list(self.attributes))
#         self.attr_to_idx = {attr: idx for idx, attr in enumerate(self.attributes)}

#         # For each image, determine its attributes (using directory name)
#         self.image_attrs = []
#         for img_path in self.images:
#             attr_name = os.path.basename(os.path.dirname(img_path))
#             attr_vector = torch.zeros(len(self.attributes))
#             attr_vector[self.attr_to_idx[attr_name]] = 1.0
#             self.image_attrs.append(attr_vector)

#         print(f"Found {len(self.attributes)} attributes and {len(self.images)} images")

#     def __len__(self):
#         return len(self.images)

#     def __getitem__(self, idx):
#         img_path = self.images[idx]
#         image = Image.open(img_path).convert('RGB')
#         attrs = self.image_attrs[idx]

#         if self.transform:
#             image = self.transform(image)

#         return image, attrs

# # Example usage
# def create_sun_dataloaders(root_dir, batch_size=32, train_split=0.8, num_workers=4):
#     # Define transformations
#     transform = transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#     ])

#     # Create dataset
#     # Choose the appropriate dataset class based on your needs
#     # dataset = SUNDataset(root_dir, transform=transform)  # For single-label classification
#     dataset = SUNMultiLabelDataset(root_dir, images_metadata=images_metadata, transform=transform)  # For multi-label classification

#     # Split into train and test sets
#     dataset_size = len(dataset)
#     train_size = int(train_split * dataset_size)
#     test_size = dataset_size - train_size
#     train_dataset, test_dataset = random_split(
#         dataset,
#         [train_size, test_size],
#         generator=torch.Generator().manual_seed(42)  # For reproducibility
#     )

#     # Create data loaders
#     train_dataloader = DataLoader(
#         train_dataset,
#         batch_size=batch_size,
#         shuffle=True,
#         num_workers=num_workers
#     )

#     test_dataloader = DataLoader(
#         test_dataset,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=num_workers
#     )

#     return train_dataloader, test_dataloader, dataset.attributes if hasattr(dataset, 'attributes') else dataset.classes

# # Example:
# train_dataloader, test_dataloader, attribute_names = create_sun_dataloaders("./1/SUN/images/")

## ViT

In [26]:
# @title Symbolic ViT
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

class AttributePredictorViT(nn.Module):
    def __init__(self, num_attributes=102):
        super().__init__()
        # Load a pretrained ViT model
        self.backbone = models.vit_b_16(pretrained=True)

        # Replace the classification head with a new one for attribute prediction
        # ViT's hidden dimension is typically 768 for base models
        hidden_size = self.backbone.heads.head.in_features
        self.backbone.heads.head = nn.Linear(hidden_size, num_attributes)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.backbone(x))

# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_vit = AttributePredictorViT(num_attributes=102).to(device)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for multi-label
optimizer = torch.optim.Adam(model_vit.parameters(), lr=0.0001)  # Lower learning rate for transformer

# Data preparation for ViT
# Note: ViT typically uses a different input size (224x224 or 384x384)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Training function
def train(model, dataloader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item()
            predicted = (outputs > 0.5).int()
            total += labels.size(0) * labels.size(1)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(dataloader)
        epoch_acc = correct / total
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

# Evaluation function
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            predicted = (outputs > 0.5).int()
            total += labels.size(0) * labels.size(1)
            correct += (predicted == labels).sum().item()

    test_loss = running_loss / len(dataloader)
    test_acc = correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

    return test_loss, test_acc


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:
# @title Training and Evaluation
train(model_vit, train_dataloader, criterion, optimizer, epochs=10)
evaluate(model_vit, val_dataloader, criterion)

Epoch 1/10, Loss: 0.2165, Accuracy: 0.8550
Epoch 2/10, Loss: 0.1694, Accuracy: 0.8629
Epoch 3/10, Loss: 0.1564, Accuracy: 0.8654
Epoch 4/10, Loss: 0.1475, Accuracy: 0.8674
Epoch 5/10, Loss: 0.1398, Accuracy: 0.8690
Epoch 6/10, Loss: 0.1322, Accuracy: 0.8704
Epoch 7/10, Loss: 0.1244, Accuracy: 0.8715
Epoch 8/10, Loss: 0.1167, Accuracy: 0.8721
Epoch 9/10, Loss: 0.1100, Accuracy: 0.8724
Epoch 10/10, Loss: 0.1051, Accuracy: 0.8725
Test Loss: 0.1779, Test Accuracy: 0.8614


(0.1778793211704419, 0.8613851259205383)

In [126]:
def predict_attributes(model, image_path, transform):
    from PIL import Image

    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_tensor)
        predicted_attributes = (output)  # Binarize to 0/1

    return predicted_attributes

========================================================================

In [127]:
predicted = predict_attributes(model_vit, '../test_images/access_road/sun_ahsirvbzddqizrxd.jpg', transform)

In [128]:
print((predicted[0]))

tensor([5.9734e-03, 7.4273e-01, 9.6145e-01, 1.1458e-01, 1.4784e-02, 1.0612e-01,
        9.5110e-02, 4.9456e-03, 1.0211e-01, 6.9229e-04, 2.2482e-04, 9.7374e-04,
        1.4621e-04, 2.3633e-03, 3.6206e-03, 3.9846e-03, 1.3468e-03, 1.7799e-03,
        3.6996e-03, 1.5969e-03, 2.5863e-04, 7.9773e-04, 1.9799e-03, 2.5594e-01,
        7.0894e-03, 2.2080e-04, 3.1911e-04, 1.3924e-02, 3.3323e-04, 5.6394e-04,
        2.9784e-04, 1.7219e-03, 9.7940e-04, 2.7881e-03, 3.6101e-04, 2.3289e-03,
        3.0986e-02, 1.1113e-03, 2.0421e-03, 2.2363e-03, 9.1894e-01, 5.2473e-01,
        9.4162e-01, 5.8994e-01, 9.9090e-01, 8.2765e-01, 1.1769e-02, 6.3314e-01,
        7.8331e-01, 5.9926e-03, 2.7297e-04, 2.4185e-03, 3.0272e-03, 3.3442e-01,
        2.5894e-03, 7.1942e-04, 1.3867e-03, 1.5570e-03, 3.5362e-03, 3.1773e-03,
        2.0758e-02, 1.5256e-02, 1.5415e-01, 5.7359e-03, 1.2457e-03, 6.1586e-04,
        9.0515e-04, 4.7326e-03, 1.8360e-03, 1.4697e-03, 1.8509e-03, 9.4468e-03,
        1.0100e-03, 5.3198e-04, 9.3906e-

In [ ]:
# @title ignore
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

class AttributePredictor(nn.Module):
    def __init__(self, num_attributes=102, extract_features=False):
        super().__init__()
        # Load Vision Transformer model with proper weights
        self.backbone = models.vit_b_16(weights='ViT_B_16_Weights.DEFAULT')
        self.hidden_dim = self.backbone.hidden_dim  # Usually 768 for ViT-B/16

        # Replace classification head with our multi-label head
        self.backbone.heads = nn.Linear(self.hidden_dim, num_attributes)
        self.sigmoid = nn.Sigmoid()
        self.extract_features = extract_features

    def forward(self, x):
        if not self.extract_features:
            # Normal prediction mode - full forward pass with sigmoid activation
            x = self.backbone(x)
            return self.sigmoid(x)
        else:
            # Feature extraction mode - get the features before classification head
            # Process input and get patch embeddings
            x = self.backbone.conv_proj(x)
            x = x.flatten(2).transpose(1, 2)
            x = self.backbone._process_input(x)

            # Add class token and position embedding
            cls_token = self.backbone.class_token.expand(x.shape[0], -1, -1)
            x = torch.cat([cls_token, x], dim=1)
            x = x + self.backbone.encoder.pos_embedding

            # Pass through transformer encoder
            x = self.backbone.encoder.dropout(x)
            x = self.backbone.encoder.layers(x)

            # Return the [CLS] token features only
            return x[:, 0]

# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AttributePredictor(num_attributes=102).to(device)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for multi-label
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Data preparation with appropriate transforms for ViT
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Training function
def train(model, dataloader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item()
            predicted = (outputs > 0.5).int()
            total += labels.size(0) * labels.size(1)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(dataloader)
        epoch_acc = correct / total
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

# Evaluation function
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            predicted = (outputs > 0.5).int()
            total += labels.size(0) * labels.size(1)
            correct += (predicted == labels).sum().item()

    test_loss = running_loss / len(dataloader)
    test_acc = correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

    return test_loss, test_acc

# Feature extraction function
def extract_features(model, image_path, transform):
    from PIL import Image

    # Create a copy of the model and set to eval mode
    model.eval()

    # Set feature extraction mode off - we'll extract at the right point manually
    model.extract_features = False

    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        # Use a hook to capture the output of the transformer encoder
        features = None

        def hook_fn(module, input, output):
            nonlocal features
            # Capture the [CLS] token (first token)
            features = output[:, 0].detach()

        # Register the hook on the last layer of the encoder
        hook = model.backbone.encoder.layers[-1].register_forward_hook(hook_fn)

        # Forward pass through the model
        _ = model(image_tensor)  # We'll capture the output with our hook

        # Remove the hook
        hook.remove()

    return features.cpu().numpy()

# Inference for a single image
def predict_attributes(model, image_path, transform):
    from PIL import Image

    # Ensure model is in prediction mode
    model.extract_features = False
    model.eval()

    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_tensor)
        predicted_attributes = (output > 0.5).int()  # Binarize to 0/1

    return predicted_attributes

# Example usage:
train(model, train_dataloader, criterion, optimizer, epochs=10)
# test_loss, test_acc = evaluate(model, test_dataloader, criterion)
# predicted_attrs = predict_attributes(model, "path/to/image.jpg", transform)
# feature_vector = extract_features(model, "/content/1/SUN/images/a/abbey/sun_aacphuqehdodwawg.jpg", transform)

In [33]:
# @title Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [90]:
torch.save(model_vit, 'vit_symbolic_true.pth')

In [91]:
!cp vit_symbolic_true.pth /content/drive/MyDrive/

In [ ]:
# @title 1-NN Dense Features
import numpy as np
from numpy.linalg import norm # Import norm specifically for clarity

def cosine_similarity(vec1, vec2):
    """
    Calculates the cosine similarity between two vectors.

    Args:
        vec1 (list or np.ndarray): The first vector.
        vec2 (list or np.ndarray): The second vector.

    Returns:
        float: The cosine similarity between the two vectors.
               Returns 0 if either vector has zero magnitude.
    """
    # Convert to numpy arrays if they aren't already
    v1 = np.array(vec1)
    v2 = np.array(vec2)

    # Calculate dot product
    dot_product = np.dot(v1, v2)

    # Calculate L2 norms (magnitudes)
    norm_v1 = norm(v1)
    norm_v2 = norm(v2)

    # Avoid division by zero
    if norm_v1 == 0 or norm_v2 == 0:
        return 0.0

    # Calculate cosine similarity
    similarity = dot_product / (norm_v1 * norm_v2)

    return similarity

def retrieve_nearest_case_cosine(query_vector, case_base):
    """
    Retrieves the nearest case from the case base using cosine similarity.

    Args:
        query_vector (list): The vector representing the query.
        case_base (list): A list of dictionaries, where each dictionary
                          represents a case and has an 'attributes' key
                          containing a vector.

    Returns:
        dict: The case from case_base with the highest cosine similarity
              to the query_vector, or None if case_base is empty.
    """
    best_case = None
    best_score = -1.0 # Cosine similarity ranges from -1 to 1

    # Convert query_vector to numpy array once outside the loop for efficiency
    query_np = np.array(query_vector)

    for case in case_base:
        # Ensure case attributes are also treated as numpy arrays
        case_attributes_np = np.array(case['features_vector'])

        # Calculate cosine similarity
        # Pass numpy arrays directly to the helper function
        sim = cosine_similarity(query_np, case_attributes_np)

        # Update best case if current similarity is higher
        if sim > best_score:
            best_score = sim
            best_case = case

    return best_case

nearest = retrieve_nearest_case_cosine(features, images_deep_features)
print(f"Nearest case based on cosine similarity: {nearest}")

In [58]:
# @title 1-NN Symbolic AI
import numpy as np
from numpy.linalg import norm # Import norm specifically for clarity

def cosine_similarity(vec1, vec2):
    """
    Calculates the cosine similarity between two vectors.

    Args:
        vec1 (list or np.ndarray): The first vector.
        vec2 (list or np.ndarray): The second vector.

    Returns:
        float: The cosine similarity between the two vectors.
               Returns 0 if either vector has zero magnitude.
    """
    # Convert to numpy arrays if they aren't already
    v1 = np.array(vec1)
    v2 = np.array(vec2)

    # Calculate dot product
    dot_product = np.dot(v1, v2)

    # Calculate L2 norms (magnitudes)
    norm_v1 = norm(v1)
    norm_v2 = norm(v2)

    # Avoid division by zero
    if norm_v1 == 0 or norm_v2 == 0:
        return 0.0

    # Calculate cosine similarity
    similarity = dot_product / (norm_v1 * norm_v2)

    return similarity

def retrieve_nearest_case_cosine(query_vector, case_base):
    """
    Retrieves the nearest case from the case base using cosine similarity.

    Args:
        query_vector (list): The vector representing the query.
        case_base (list): A list of dictionaries, where each dictionary
                          represents a case and has an 'attributes' key
                          containing a vector.

    Returns:
        dict: The case from case_base with the highest cosine similarity
              to the query_vector, or None if case_base is empty.
    """
    best_case = None
    best_score = -1.0 # Cosine similarity ranges from -1 to 1

    # Convert query_vector to numpy array once outside the loop for efficiency
    query_np = query_vector.cpu().numpy() if hasattr(query_vector, 'cpu') else np.array(query_vector)

    for case in case_base:
        # Ensure case attributes are also treated as numpy arrays
        case_attributes_np = np.array(case['labels_attr'])

        # Calculate cosine similarity
        # Pass numpy arrays directly to the helper function
        sim = cosine_similarity(query_np, case_attributes_np)

        # Update best case if current similarity is higher
        if sim > best_score:
            best_score = sim
            best_case = case

    return best_case


In [76]:
print(predicted[0].cpu().numpy())

[0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]


In [83]:
# print(images_metadata[0])
nearest, confidence = predict_from_knn(predicted[0], images_metadata, k = 5)
print(f"Nearest case based on cosine similarity: {nearest}, confidence: {confidence}")

Nearest case based on cosine similarity: planetarium_outdoor, confidence: 0.880941470823506


In [107]:
# @title KNN Symbolic AI
import numpy as np
from numpy.linalg import norm
import heapq  # For efficient k-smallest elements handling

def cosine_similarity(vec1, vec2):
    """
    Calculates the cosine similarity between two vectors.

    Args:
        vec1 (list or np.ndarray): The first vector.
        vec2 (list or np.ndarray): The second vector.

    Returns:
        float: The cosine similarity between the two vectors.
               Returns 0 if either vector has zero magnitude.
    """
    # Convert to numpy arrays if they aren't already
    v1 = np.array(vec1)
    v2 = np.array(vec2)

    # Calculate dot product
    dot_product = np.dot(v1, v2)

    # Calculate L2 norms (magnitudes)
    norm_v1 = norm(v1)
    norm_v2 = norm(v2)

    # Avoid division by zero
    if norm_v1 == 0 or norm_v2 == 0:
        return 0.0

    # Calculate cosine similarity
    similarity = dot_product / (norm_v1 * norm_v2)

    return similarity

def retrieve_k_nearest_cases_cosine(query_vector, case_base, k=3):
    """
    Retrieves the k nearest cases from the case base using cosine similarity.

    Args:
        query_vector (list or np.ndarray): The vector representing the query.
        case_base (list): A list of dictionaries, where each dictionary
                          represents a case and has a 'labels_attr' key
                          containing a vector.
        k (int): The number of nearest neighbors to retrieve.

    Returns:
        list: A list of (case, similarity_score) tuples containing the k cases
              with highest cosine similarity to the query_vector, sorted
              in descending order of similarity.
    """
    if not case_base:
        return []

    # Cap k to the size of the case base
    k = min(k, len(case_base))

    # Convert query_vector to numpy array once outside the loop for efficiency
    query_np = query_vector.cpu().numpy() if hasattr(query_vector, 'cpu') else np.array(query_vector)

    # Calculate similarities for all cases first
    all_similarities = []

    for i, case in enumerate(case_base):
        # Ensure case attributes are also treated as numpy arrays
        case_attributes_np = np.array(case['labels_attr'])

        # Calculate cosine similarity
        sim = cosine_similarity(query_np, case_attributes_np)

        # Store similarity with case index
        # We use negative similarity for the min-heap to function as a max-heap
        all_similarities.append((-sim, i))

    # Get k nearest neighbors using heapq
    k_nearest_indices = heapq.nsmallest(k, all_similarities)

    # Format results as [(case, similarity)]
    result = [(case_base[idx], -sim) for sim, idx in k_nearest_indices]

    return result

def predict_from_knn(query_vector, case_base, k=3):
    """
    Makes a prediction based on the k nearest neighbors.

    Args:
        query_vector (list or np.ndarray): The vector representing the query.
        case_base (list): A list of dictionaries, where each dictionary
                          represents a case with 'labels_attr' and 'label' keys.
        k (int): The number of nearest neighbors to consider.

    Returns:
        tuple: (predicted_label, confidence_score)
               The predicted label is based on majority voting.
               The confidence score is the average similarity of the k neighbors.
    """
    neighbors = retrieve_k_nearest_cases_cosine(query_vector, case_base, k)

    if not neighbors:
        return None, 0.0

    # Count votes for each label
    votes = {}
    total_similarity = 0.0

    for case, similarity in neighbors:
        label = case.get('label')
        if label not in votes:
            votes[label] = 0
        votes[label] += 1
        total_similarity += similarity

    # Find label with most votes
    predicted_label = max(votes.items(), key=lambda x: x[1])[0]

    # Calculate confidence score (average similarity)
    confidence_score = total_similarity / len(neighbors)

    return predicted_label, confidence_score

# Example usage
# Create a case base with attribute vectors and labels
# case_base = [
#     {'labels_attr': [1, 0, 1, 0], 'label': 'A'},
#     {'labels_attr': [0, 1, 0, 1], 'label': 'B'},
#     {'labels_attr': [1, 1, 0, 0], 'label': 'A'},
#     {'labels_attr': [0, 0, 1, 1], 'label': 'B'},
#     {'labels_attr': [1, 0, 0, 1], 'label': 'C'},
# ]

# Query vector
# query = [0.9, 0.1, 0.8, 0.2]

# # Get 3 nearest neighbors
# neighbors = retrieve_k_nearest_cases_cosine(query, case_base, k=3)
# print("Top 3 neighbors:")
# for case, similarity in neighbors:
#     print(f"Case: {case}, Similarity: {similarity:.4f}")

# # Make a prediction
# prediction, confidence = predict_from_knn(query, case_base, k=3)
# print(f"Prediction: {prediction}, Confidence: {confidence:.4f}")

In [ ]:
predicted = predict_attributes(model_vit, '../test_images/access_road/sun_ahsirvbzddqizrxd.jpg', transform)

In [105]:
# @title Get All Test Files
# Option 1: Using os module
import os

# Option 2: Using pathlib (more modern approach)
from pathlib import Path

def traverse_directory_pathlib(directory_path):
    """
    Traverse all files in the specified directory and its subdirectories using pathlib.

    Args:
        directory_path (str): Path to the directory to traverse

    Returns:
        list: A list of Path objects for all files found
    """
    all_files = []
    directory = Path(directory_path)

    try:
        # Check if the directory exists
        if not directory.exists():
            print(f"Directory does not exist: {directory_path}")
            return all_files

        # Recursively iterate through all files
        for file_path in directory.glob('**/*'):
            if file_path.is_file():
                all_files.append(file_path)

                # Print file information
                # print(f"Found file: {file_path}")
                # print(f"  - Size: {file_path.stat().st_size} bytes")
                # print(f"  - Type: {file_path.suffix}")

    except Exception as e:
        print(f"Error traversing directory: {e}")

    return all_files

# Example usage
if __name__ == "__main__":
    # Replace "../test_images" with your actual directory path if needed
    directory_path = "../test_images"

    files_pathlib = traverse_directory_pathlib(directory_path)


In [106]:
print(files_pathlib[34])

../test_images/cathedral_outdoor/sun_atadimdgdmjvnizz.jpg


In [129]:
total_examples = 0
matches = 0
for f in files_pathlib:
  # print("file_path: ", f)
  real_category = str(f).split('/')[-2]
  predicted = predict_attributes(model_vit, f, transform)
  nearest, confidence = predict_from_knn(predicted[0], images_metadata, k = 20)
  if (real_category == nearest): matches += 1
  # print(f"Nearest case based on cosine similarity: {nearest}, confidence: {confidence}")
  total_examples += 1
  # if (total_examples == 10): break
  print(total_examples)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [130]:
print(matches)

260
